# HBV model - calibration

Andrew Ireson, 19 March 2022

In [ ]:
import numpy as np
import matplotlib.pyplot as pl
import pandas as pd

import sys
sys.path.insert(1,'lib')

# Select here which model to run:
sys.path.insert(1,'hbv')

import model_library as model

import time

In [ ]:
# Pick basin:
basin='Banff basin'

In [ ]:
# Initial parameter guesses
par_values={}
par_values['TT']=0
par_values['C0']=1
par_values['ETF']=0.1
par_values['LP']=0.3
par_values['FC']=250
par_values['beta']=2
par_values['FRAC']=0.7
par_values['K1']=0.05
par_values['alpha']=1.5
par_values['K2']=0.01
par_values['UBAS']=1
par_values['PM']=1.

In [ ]:
# Load observed streamflow
Qobs=model.obs_streamflow(basin)

In [ ]:
# Run the model with user specified values
fluxUC,state,forcing=model.StartRun(basin,par_values)

In [ ]:
# Plot observed and simulated streamflow
start='1998'
end='2001'

# Plot performance
pl.figure(figsize=(10,5))
pl.plot(fluxUC['Q_cms'][start:end],'-',color='royalblue',label='Simulated')
pl.plot(Qobs[start:end],'.',color='sienna',label='Observed')
pl.ylabel('Streamflow (cms)',fontsize=13); pl.grid()
pl.legend(fontsize=13)

In [ ]:
# Generate parameters bounds for optimization/uncertainty analysis
par_bounds={}                        # Default value
par_bounds['TT']=(-4,4)              # 0.
par_bounds['C0']=(0,10)              # 1.
par_bounds['ETF']=(0,1)              # 0.1
par_bounds['LP']=(0,1)               # 0.3
par_bounds['FC']=(50,500)            # 250
par_bounds['beta']=(1,3)             # 2.
par_bounds['FRAC']=(0.1,0.9)         # 0.7
par_bounds['K1']=(0.05,1)            # 0.05
par_bounds['alpha']=(1,3)            # 1.5
par_bounds['K2']=(0,0.05)            # 0.01
par_bounds['UBAS']=(1,3)             # 1.
par_bounds['PM']=(0.5,2)             # 1.

In [ ]:
# Specify dates for calibration and validation
dates={}
dates['start_spin']='1998'
dates['start_calib']='2000'
dates['end_calib']='2000'
dates['start_validation']=dates['end_calib']
dates['end_validation']='2001'

In [ ]:
# list of parameters to be optimized, and associated starting values
pn=[k for k in par_values]
pv=[par_values[k] for k in par_values]
pn=pn[:-1]
pv=pv[:-1]
metric='RMSE'

# Run optimization
par_values,pv=model.run_optimization(basin,dates,metric,par_bounds,par_values,pn,pv)

In [ ]:
# Run model with optimized parameters and save outputs:
flux,state,forcing=model.StartRun(basin,par_values)

In [ ]:
calib=-model.eval_metric(Qobs[dates['start_calib']:dates['end_calib']].values.squeeze(),flux['Q_cms'][dates['start_calib']:dates['end_calib']].values,'NSE')
valid=-model.eval_metric(Qobs[dates['start_validation']:dates['end_validation']].values.squeeze(),flux['Q_cms'][dates['start_validation']:dates['end_validation']].values,'NSE')
print('%s (calibration): %.4f'%(metric,calib))
print('%s (validation): %.4f'%(metric,valid))

In [ ]:
# Plot calibration performance
start=dates['start_spin']
end=dates['end_validation']
pl.figure(figsize=(10,5))
pl.plot(Qobs[start:end],'.',label='Observed')
pl.plot(fluxUC['Q_cms'][start:end],label='Uncalibrated simulation')
pl.plot(flux['Q_cms'][start:end],label='Calibrated simulation')
pl.legend(fontsize=13)
pl.ylabel('Streamflow (cms)',fontsize=13); pl.grid()